In [1]:
%pip install ollama

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
Using cached anyio-4.10.0-py3-none-any.whl (107 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)

   ------ --------------------------------- 1/6 [h11]
   ------ --------------------------------- 1/6 [h11]
   ------ --------------------------------- 1/6 [h11]
   ------ --------------------------------- 1/6 [h11]
   ------------- -------------------------- 2/6 [httpcore]
   ------------- -------------------------- 2/6 [httpcore]
   ------------- -------------------------- 2/6 [httpcore]
   ------------- -----------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!ollama list

NAME         ID              SIZE      MODIFIED    
gemma3:1b    8648f39daa8f    815 MB    10 days ago    
qwen3:4b     e55aed6fe643    2.5 GB    10 days ago    


In [ ]:
def build_prompt(query_book, query_description, query_vector, examples, n_shots=0):
    # Base context
    context = """You help recommend books to teenagers. You will do this by analyzing the emotion context of the book description and comparing it to the emotion vectors of what different age groups of teenagers prefer in their book descriptions. Here are the average emotion vectors of books that each age group enjoyed:

"12-13": {'Anger': 5.928, 'Anticipation': 11.774, 'Disgust': 3.710, 'Fear': 10.412, 'Joy': 15.148, 'Sadness': 6.429, 'Surprise': 5.140, 'Trust': 16.259},

"14-15": {'Anger': 6.907, 'Anticipation': 10.844, 'Disgust': 4.491, 'Fear': 11.033, 'Joy': 14.164, 'Sadness': 7.528, 'Surprise': 4.784, 'Trust': 15.062},

"16-17": {'Anger': 7.961, 'Anticipation': 12.758, 'Disgust': 4.934, 'Fear': 13.352, 'Joy': 15.926, 'Sadness': 9.764, 'Surprise': 5.474, 'Trust': 16.686},

"18+": {'Anger': 6.049, 'Anticipation': 10.697, 'Disgust': 3.925, 'Fear': 10.116, 'Joy': 14.021, 'Sadness': 7.400, 'Surprise': 4.871, 'Trust': 16.471}
"""
    
    # Add few-shot examples
    shots = ""
    for i in range(min(n_shots, len(examples))):
        ex = examples[i]
        shots += f"""
Here is an example problem and answer:
Question: What age group would you recommend the book “{ex['title']}”? Here is the book description: "{ex['description']}" Here is the normalized emotion vector of the description: {ex['vector']}
Answer: {ex['answer']}
"""
    
    # Add the actual query
    query = f"""
Here is the question the user has provided:
What age group would you recommend the book “{query_book}”? Here is the book description: "{query_description}" Here is the normalized emotion vector of the description: {query_vector}
Respond only with the age group.
"""
    
    return context + shots + query

In [4]:
import ollama

def run_ollama(prompt, model="gemma3:1b"):
    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response["message"]["content"].strip()

In [ ]:
import random
import pandas as pd

examples_csv = pd.read_csv("correct_cosine_with_vectors.csv")

def age_to_group(age):
    if 12 <= age < 14:
        return "12-13"
    elif 14 <= age < 16:
        return "14-15"
    elif 16 <= age < 18:
        return "16-17"
    else:
        return "18+"

few_shot_examples = []
for _, row in examples_csv.iterrows():
    few_shot_examples.append({
        "title": row["title"],
        "description": row["description"],
        "vector": row["emotion_vector"],
        "answer": age_to_group(row["age"])
    })

models = ['gemma3:1b', 'qwen3:4b']
shot_settings = [0, 1, 3, 5]

query_rows = examples_csv.sample(5)

results = []

for _, query_row in query_rows.iterrows():
    query_book = query_row["title"]
    query_description = query_row["description"]
    query_vector = query_row["emotion_vector"]

    for model in models:
        for n_shots in shot_settings:
            examples = random.sample(few_shot_examples, min(n_shots, len(few_shot_examples)))
            for i in range(5): #arbitrary number to see any variance in LLM response with the same prompt
                prompt = build_prompt(query_book, query_description, query_vector, examples, n_shots)
                answer = run_ollama(prompt, model=model)
                
                results.append({
                    "model": model,
                    "shots": n_shots,
                    "book": query_book,
                    "answer": answer
                })


df = pd.DataFrame(results)
df.to_csv("ollama_fewshot_results.csv", index=False)
